<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/Fake_News_Classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
import pandas as pd
import os

In [4]:
!unzip /content/drive/MyDrive/train_LSTM.zip


Archive:  /content/drive/MyDrive/train_LSTM.zip
  inflating: train_LSTM.csv          


In [5]:
df=pd.read_csv('/content/train_LSTM.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df.reset_index(drop=True,inplace=True)

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
X=df.drop('label',axis=1)
y=df['label']

In [15]:
X.shape

(18285, 4)

In [37]:
X.drop('id',axis=1,inplace=True)

In [16]:
import tensorflow as tf


In [19]:
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
messages1=X.copy()

## DATA CLEANING


In [27]:
def tolower(text):
  return text.lower()

In [39]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(tolower)

In [41]:
messages1.head()

,title,author,text
0,house dem aide: we didn’t even see comey’s let...,darrell lucus,house dem aide: we didn’t even see comey’s let...
1,"flynn: hillary clinton, big woman on campus - ...",daniel j. flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnews.com,"why the truth might get you fired october 29, ..."
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [42]:
## REMOVING HTML TAGS
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [44]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_html_tags)

In [45]:
## removing url
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [46]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_url)

In [48]:
## HANDLING PUNCTUATION
import string
def remove_punc(text):
  for i in string.punctuation:
    text=text.replace(i,'')
  return text


In [49]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_punc)

In [50]:
messages1.head()

,title,author,text
0,house dem aide we didn’t even see comey’s lett...,darrell lucus,house dem aide we didn’t even see comey’s lett...
1,flynn hillary clinton big woman on campus bre...,daniel j flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnewscom,why the truth might get you fired october 29 2...
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [51]:
## REMOVING STOPWORDS
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
def remove_stopwords(text):
  words=text.split()
  for i,word in enumerate(words):
    if word in stopwords.words('english'):
      words[i]=''
  return ' '.join(words)

In [53]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_stopwords)

In [55]:
messages1['title'][2]

'  truth might get  fired'

In [56]:
def remove_extra_spaces(sentence):

    cleaned_sentence = ' '.join(sentence.split()).strip()

    return cleaned_sentence

In [57]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_extra_spaces)

In [58]:
messages1['title'][2]

'truth might get fired'

In [59]:
messages1.to_csv('cleaned_data.csv')